# Step 1. Import Libraries and Load the Data
Import all the required modules. Then read in required files.

In [1]:
# Built in libraries
import json
import random
import pickle
import tkinter
from tkinter import *
import textwrap

# 3rd party libraries
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.models import load_model
model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# Step 2. Define Processing Functions
Define functions necessary to process the user's message before inputting it into the model, as well as functions necessary to make predictions and get a response.

In [2]:
# create a data structure to hold user context
context = []

# tokenize the sentence, then lemmaztize and lower each word and remove duplicates
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # initializing bag of words
    bag = [0]*len(words)  
    # set the index of a word in the bag of words to 1, if the respective word is found in the sentence
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.70
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": r[1]})
    return return_list

# the "ints" parameter is the output of the predict_class() function
def getResponse(ints, intents_json):
    # Check if the sentence is part of a context or has an intent
    if context:
        tag = context.pop()[0]
    elif not ints:
        tag = 'noanswer'
    else:
        # null threshold is the accuracy predicted for sentences making no sense
        # (ie, numbers, different languages, troll messages, etc)
        if (NULL_THRESHOLD-0.000001<ints[0]['probability']<NULL_THRESHOLD+0.000001):
            tag = 'noanswer'
        else:
            tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            if i['context'] != [""]:
                context.append(i['context'])
    return result

# null threshold is the accuracy predicted for sentences making no sense
# (ie, numbers, different languages, troll messages, etc)
# this probability is outputted below
NULL_THRESHOLD = predict_class("12345")[0]['probability']

## Examples
Below are some examples for how the chatbot responds to messages.

In [3]:
msg = "Hello"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: Hello
Bot: Hi there, how can I help?


In [4]:
msg = "How does Zelle work?"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: How does Zelle work?
Bot: Zelle is an easy way to send money directly between almost any U.S. bank accounts typically within minutes. With just an
email address or mobile phone number, you can quickly, safely and easily send  and receive money with more people, regardless
of where they bank.


In [5]:
msg = "Does Zelle cost anything?"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: Does Zelle cost anything?
Bot: Zelle doesn't charge a fee to send or receive money. We recommend confirming with your bank or credit union that there
are no additional fees.


In [6]:
msg = "How long does it take to send money?"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: How long does it take to send money?
Bot: Money sent with Zelle is typically available to an enrolled recipient within minutes. If it has been more than three
days, please confirm you have fully enrolled your Zelle profile, and that you entered the correct email address or number. If
you're using Zelle through your bank or credit union's app or website, please check with them for help.


In [7]:
msg = "Can I use Zelle on my computer instead of my phone?"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: Can I use Zelle on my computer instead of my phone?
Bot: If your bank or credit union offers Zelle, you may be able to use their online banking site to send and receive money
without a mobile device. If your bank or credit union does not offer Zelle, then you must have a smartphone in order to
access the Zelle app to send and receive money.


In [8]:
# An examply of a contextual conversation
msg = "Can I talk to an employee?"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))
msg = "123-456-7890"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: Can I talk to an employee?
Bot: Sure, I'll put you in touch with a team member. Please provide a phone number you would like to be called on.
You: 123-456-7890
Bot: Thank you, a team member will call you shortly.


In [9]:
msg = "Thanks, bye"
ints = predict_class(msg)
res = getResponse(ints, intents)
print(textwrap.fill("You: " + msg, 125))
print(textwrap.fill("Bot: " + res, 125))

You: Thanks, bye
Bot: My pleasure.


# Step 3: Creating GUI
Create the chat window and all of its components, including a function for it to send and recieve messages with the use.

In [10]:
def send(self=None):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 

        ints = predict_class(msg)
        res = getResponse(ints, intents)

        ChatBox.insert(END, "Bot: " + res + '\n\n')           

        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

# initialize Tkinter interpreter and create the root window
root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

# Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial", wrap=WORD)

# chatbox is set to read-only (for now)
ChatBox.config(state=DISABLED)

# Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

# Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )
root.bind("<Return>", send)

# Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial", wrap=WORD)

# Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Send the introduction message
intro = "Welcome to Zelle chatbot! I can answer basic questions regarding Zelle, or I can have a team member call you if needed. What would you like help with today?"
ChatBox.config(state=NORMAL)
ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
ChatBox.insert(END, "Bot: " + intro + '\n\n')         
ChatBox.config(state=DISABLED)
ChatBox.yview(END)

# run the event loop
root.mainloop()